## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [1]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.12 ms ± 6.48 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.4 ms ± 19.6 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.4 ms ± 28.1 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


# Inicialización
En el bloque de código original no se especifica el tipo de dato usado, por lo que asumo que durante el resto de la práctica, a no ser que se pida, se usará el tipo por defecto para punto flotante (float64)

In [2]:
import cupy as cp
from cupyx.profiler import benchmark

#Inicializacion
cuda_device = 0
devices = cp.cuda.runtime.getDeviceCount()
assert devices > 0, 'No CUDA-powered device found'
print('Found {} CUDA devices'.format(devices))
cp.cuda.runtime.setDevice(cuda_device)
specs = cp.cuda.runtime.getDeviceProperties(cuda_device)
name = specs['name'].decode()
print('Using CuPy with : {}'.format(name))

Found 1 CUDA devices
Using CuPy with : NVIDIA GeForce GTX 1080


In [3]:
from numba import vectorize, float64, cuda

cuda.detect()

Found 1 CUDA devices
id 0    b'NVIDIA GeForce GTX 1080'                              [SUPPORTED]
                      Compute Capability: 6.1
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-574ca04c-bd1f-2193-bb43-192268bb8508
                                Watchdog: Disabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

# Apartado 3.2
## Ejercicio A

Vamos a usar los arrays de *numpy* creados en el primer bloque de código, copiandolos a GPU con `CuPy.asarray` para poder calcular $z$ en GPU. Todo esto debe de hacerse dentro de la misma función para que se mida correctamente por la función `benchmark`. No se tendrá en cuenta el tiempo de creación de los arrays de *numpy*

In [4]:
def grade2_cupy_copy(x, y, a, b, c):
    # los arrays x e y ya están creados en CPU
    x_gpu = cp.asarray(x)
    y_gpu = cp.asarray(y)
    return a*x_gpu**2 + b*y_gpu + c

c_gpu = grade2_cupy_copy(a_cpu, b_cpu, a, b, c)

assert c_cpu.all() == c_gpu.all(), 'CPU y GPU tienen resultados diferentes'

execution_gpu = benchmark(grade2_cupy_copy, (a_cpu, b_cpu, a, b, c,), n_repeat=10, n_warmup=2)
gpu_avg_time = np.average(execution_gpu.gpu_times) * 1e3  # convert to ms
print(f"Elapsed time grade2_cupy_copy: {gpu_avg_time:.3f} ms")

Elapsed time grade2_cupy_copy: 17.208 ms


Ahora, se crearán en la función los arrays dentro de GPU con la función `CuPy.random`

In [6]:
def grade2_cupy_no_copy(x_gpu, y_gpu, a, b, c):
    return a*x_gpu**2 + b*y_gpu + c

# El resultado será distinto al de las anteriores ejecuciones
x_gpu = cp.random.rand(size)
y_gpu = cp.random.rand(size)
execution_gpu = benchmark(grade2_cupy_no_copy, (x_gpu, y_gpu, a, b, c,), n_repeat=10, n_warmup=2)
gpu_avg_time = np.average(execution_gpu.gpu_times) * 1e3  # convert to ms
print(f"Elapsed time grade2_cupy_no_copy: {gpu_avg_time:.3f} ms")

Elapsed time grade2_cupy_no_copy: 4.839 ms


Como se puede apreciar, hay un *overhead* considerable por copiar las variables a GPU.

## Ejercicio B

Creamos la función `grade2_numba` que será llamada por las dos funciones pedidas (con y sin copia)

In [ ]:
@vectorize(['float64(float64, float64, float64, float64, float64)'], target = 'cuda')
def grade2_numba(x, y, a, b, c):
    return a*x**2 + b*y + c

# Función con copia
def grade2_numba_copy(x, y, a, b, c):
    return grade2_numba(x, y, a, b, c)
    
# Función sin copia
def grade2_numba_no_copy(x, y, a, b, c):
    c_gpu_numba = grade2_numba(x, y, a, b, c)
    cuda.synchronize() 
    return c_gpu_numba

# Copia manual de arrays a GPU
a_gpu = cuda.to_device(a_cpu)
b_gpu = cuda.to_device(b_cpu)

# Tiempos con benchmark
execution_gpu = benchmark(grade2_numba_copy, (a_cpu, b_cpu, a, b, c,), n_repeat=10, n_warmup=2)
gpu_avg_time = np.average(execution_gpu.gpu_times) * 1e3  # convert to ms
print(f"Elapsed time grade2_numba_copy: {gpu_avg_time:.3f} ms")

execution_gpu = benchmark(grade2_numba_no_copy, (a_gpu, b_gpu, a, b, c,), n_repeat=10, n_warmup=2)
gpu_avg_time = np.average(execution_gpu.gpu_times) * 1e3  # convert to ms
print(f"Elapsed time grade2_numba_no_copy: {gpu_avg_time:.3f} ms")

## Ejercicio C
Se ha pasado del código original, que se ejecuta secuencialmente en CPU, a [código asíncrono y concurrente](https://docs.nvidia.com/cuda/cuda-programming-guide/02-basics/asynchronous-execution.html) ejecutado en GPU. Al haber usado la función `benchmark` de *cupy*, se maneja internamente la [sincronización](https://docs.cupy.dev/en/stable/user_guide/performance.html) tanto para cuando usamos *cupy* como con *numba*.

En cuanto a los tiempos, podemos ver que, con los datos usados, no hay una gran diferencia entre usar código secuencial optimizado con *numba* o *numpy* y el código ejecutado en GPU cuando se hace copia de datos. Sin embargo, al copiar manualmente los datos a GPU con `cuda.to_device` o generarlos directamente en GPU con funciones de *cupy* como `cp.random.rand`, se obtiene una gran mejora. *Numba* se beneficia más de la copia de datos a GPU. Si vamos a la [documentacion](https://numba.pydata.org/numba-doc/dev/cuda/ufunc.html) de *ufunc* de *numba* podemos leer que *"The CUDA ufunc adds support for passing intra-device arrays (already on the GPU device) to reduce traffic over the PCI-express bus"*, que puede ser la razón por la que da mejores tiempos.

Existen estudios donde se comparan ambos paquetes como [*Exploring Numba and CuPy for GPU-Accelerated Monte Carlo Radiation Transport*](https://www.researchgate.net/publication/379125485_Exploring_Numba_and_CuPy_for_GPU-Accelerated_Monte_Carlo_Radiation_Transport).